In [1]:
import os.path
import sys

import os

# Récupère le chemin d'accès parent de sys.path[0]
parent_dir = os.path.dirname(sys.path[0])

sys.path.append(os.path.abspath(parent_dir))
import importlib
import algorithmes
importlib.reload(algorithmes)
from algorithmes import *
import random
import mesa
import bisect

In [3]:
def get_matrice_adjacence(route):
     
    # Extraire tous les arcs de la route
    arcs_voiture = set()
    for sous_route in route1:
        sous_route = [0]+sous_route+[0]
        arcs_voiture.update(set(zip(sous_route, sous_route[1:])))

    # Initialiser la matrice d'adjacence avec des zéros
    n = len(matrice_temps_Cij)
    matrice_adj = [[0]*n for _ in range(n)]

    # Mettre à jour les valeurs correspondantes à chaque arc
    for i, j in arcs_voiture:
        matrice_adj[i][j] = 1
        matrice_adj[j][i] = 1
        #matrice_adj[i][j] = matrice_temps_Cij[i][j]
        #matrice_adj[j][i] = matrice_temps_Cij[j][i]
    
    return(np.array(matrice_adj))
